In [7]:

import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt
import numpy as np

def get_dataset_from_testcase(run):
    return run[0:-2]

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/join_all'
runs = []
index_stats = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


In [10]:
epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "flatpgm" in x:
        return "FLAT_PGM"
    elif "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"


df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['epsilon'] = df['spec.index.epsilon']

print(df['join_algo'].unique())


['lsj' 'inlj' 'hash_join' 'sort_join']


In [11]:
def plot_for_dataset(dataset, join_algo):
    rows = duckdb.sql(
        " SELECT ratio, index_type, epsilon, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}' AND join_algo='{join_algo}'"
        " GROUP BY ratio, join_algo, index_type, epsilon"
        " ORDER BY ratio"
    ).df()
    rows = rows.pivot(index=['ratio', 'epsilon'], values='v', columns=['index_type'])
    return rows


# INLJ (Raw Lookup)

In [12]:
for dataset in ['fb', 'wiki', 'books', 'osm', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal']:
    display(dataset, plot_for_dataset(dataset, 'inlj'))

'fb'

index_type         BTREE   FLAT_PGM        PGM
ratio epsilon                                 
1     256.0    53.067202  37.117513  39.427941
      1024.0   54.454780  37.869348  38.800283
      4096.0   53.186117  37.578133  42.717257
10    256.0     6.178218   5.283925   4.912952
      1024.0    6.049074   5.495730   5.297466
      4096.0    5.819146   5.488276   5.850861
100   256.0     1.664028   1.426762   1.084884
      1024.0    1.130245   1.495151   1.199611
      4096.0    1.093045   1.685457   1.413768
1000  256.0     0.259721   0.368418   0.352090
      1024.0    0.398953   0.456532   0.491207
      4096.0    0.510789   0.710390   0.787943

'wiki'

index_type         BTREE   FLAT_PGM        PGM
ratio epsilon                                 
1     256.0    23.540610  14.672424  15.039791
      1024.0   22.996426  15.520501  15.998415
      4096.0   21.834449  16.163976  16.979243
10    256.0     5.059676   3.857612   3.505500
      1024.0    5.454602   3.812501   3.834758
      4096.0    4.802448   4.033567   4.059837
100   256.0     0.875864   0.929297   1.013105
      1024.0    0.822400   0.913240   0.840494
      4096.0    0.808017   1.058304   0.964979
1000  256.0     0.222480   0.236809   0.223985
      1024.0    0.276095   0.314903   0.293112
      4096.0    0.292301   0.411057   0.404468

'books'

...

index_type         BTREE   FLAT_PGM        PGM
ratio epsilon                                 
1     256.0    52.301702  32.761730  36.662256
      1024.0   55.245917  34.660910  36.347464
      4096.0   53.443362  35.269000  37.755019
10    256.0     5.985692   4.857902   4.701640
      1024.0    6.570872   4.613718   5.260598
      4096.0    6.335806   5.108075   5.537722
100   256.0     1.207014   1.230709   1.012684
      1024.0    1.071655   1.174059   1.513845
      4096.0    1.442828   1.506006   1.740371
1000  256.0     0.252391   0.324038   0.272613
      1024.0    0.376210   0.395045   0.372605
      4096.0    0.488717   0.563688   0.547029

'normal'

index_type         BTREE   FLAT_PGM        PGM
ratio epsilon                                 
1     256.0    54.062095  29.174066  29.698941
      1024.0   56.033252  31.476954  32.682569
      4096.0   53.901132  33.892302  35.679257
10    256.0     6.727578   4.540253   3.945524
      1024.0    6.414867   5.032417   4.791668
      4096.0    6.607661   5.459716   5.019567
100   256.0     1.192461   1.047128   0.938083
      1024.0    1.462578   1.611624   1.569407
      4096.0    1.394548   1.788865   1.924672
1000  256.0     0.243950   0.294238   0.312992
      1024.0    0.424038   0.402624   0.360370
      4096.0    0.538436   0.658574   0.732859

'lognormal'

index_type         BTREE   FLAT_PGM        PGM
ratio epsilon                                 
1     256.0    54.251634  29.030724  33.305641
      1024.0   55.068840  31.849229  32.894151
      4096.0   52.642450  34.891457  37.297779
10    256.0     6.138937   4.525924   4.297023
      1024.0    6.585124   5.154431   4.741701
      4096.0    5.840396   6.189053   6.134477
100   256.0     1.189923   1.251516   1.006116
      1024.0    1.449490   1.530472   1.235123
      4096.0    1.015438   2.252705   2.442943
1000  256.0     0.239541   0.322693   0.258540
      1024.0    0.427705   0.402910   0.373348
      4096.0    0.438325   0.727756   0.837495

# Learned Sort Join


In [6]:
for dataset in ['fb', 'wiki', 'books', 'osm', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal']:
    display(dataset, plot_for_dataset(dataset, 'lsj'))

'fb'

,index_type
ratio,epsilon


'wiki'

,index_type
ratio,epsilon


'books'

...

,index_type
ratio,epsilon


'normal'

,index_type
ratio,epsilon


'lognormal'

,index_type
ratio,epsilon
